In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-14 21:48:26--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M   366KB/s    in 3.0s    

2024-04-14 21:48:30 (366 KB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
len(text)

1115394

In [3]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda seq: ''.join(itos[num] for num in seq)

print(encode('transformer'))
print(decode(encode('transformer')))

[58, 56, 39, 52, 57, 44, 53, 56, 51, 43, 56]
transformer


In [12]:
import torch
import torch.nn as nn
data = torch.tensor(encode(text), dtype=torch.long)
print(data.dtype, data.shape)
print(data[:200])

torch.int64 torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [7]:
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when context is {context} target: {target}')

when context is tensor([18]) target: 47
when context is tensor([18, 47]) target: 56
when context is tensor([18, 47, 56]) target: 57
when context is tensor([18, 47, 56, 57]) target: 58
when context is tensor([18, 47, 56, 57, 58]) target: 1
when context is tensor([18, 47, 56, 57, 58,  1]) target: 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]) target: 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target: 58


In [10]:
torch.manual_seed(1000)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch('train')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

print('-----')


for b in range(batch_size):# batch loop
    for t in range(block_size):# time loop
        context = xb[b, :t+1]
        target = yb[b, t]

        print(f'when context is {context.tolist()} target: {target.tolist()}')


torch.Size([4, 8])
tensor([[47, 57, 46,  1, 40, 50, 53, 53],
        [56, 39, 63,  5, 57,  1, 40, 56],
        [ 1, 15, 47, 58, 47, 64, 43, 52],
        [54, 43,  6,  1, 58, 46, 53, 59]])
torch.Size([4, 8])
tensor([[57, 46,  1, 40, 50, 53, 53, 42],
        [39, 63,  5, 57,  1, 40, 56, 43],
        [15, 47, 58, 47, 64, 43, 52, 10],
        [43,  6,  1, 58, 46, 53, 59,  1]])
-----
when context is [47] target: 57
when context is [47, 57] target: 46
when context is [47, 57, 46] target: 1
when context is [47, 57, 46, 1] target: 40
when context is [47, 57, 46, 1, 40] target: 50
when context is [47, 57, 46, 1, 40, 50] target: 53
when context is [47, 57, 46, 1, 40, 50, 53] target: 53
when context is [47, 57, 46, 1, 40, 50, 53, 53] target: 42
when context is [56] target: 39
when context is [56, 39] target: 63
when context is [56, 39, 63] target: 5
when context is [56, 39, 63, 5] target: 57
when context is [56, 39, 63, 5, 57] target: 1
when context is [56, 39, 63, 5, 57, 1] target: 40
when conte

In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1110)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)#make vector from token
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_token):
        for _ in range(max_new_token):
            #make prediction
            logits, loss = self(idx)
            #take last element
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

torch.Size([256, 65])
tensor(4.5061, grad_fn=<NllLossBackward0>)


In [33]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(m.generate(idx, max_new_token=100)[0].tolist()))


y,A-!NOy,i&MtA3LxkP?l,3;&ZG$dg,yZhMjEVNYzX$bs?XWEMvsF!HkejwuQ-?CUIBLCUmOt.kgrKGRQoyIhfgy?lXYAahkJwsu


In [34]:
optimizer = torch.optim.AdamW(params=m.parameters(), lr=1e-3)

In [45]:
batch_size = 32
for steps in range(10000):

    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    print(loss.item())

2.581622838973999
2.675847053527832
2.617220163345337
2.628659248352051
2.636234760284424
2.6152658462524414
2.677347183227539
2.715876579284668
2.6202712059020996
2.6541666984558105
2.633535861968994
2.5896096229553223
2.6259653568267822
2.5684804916381836
2.5775957107543945
2.7938661575317383
2.6773183345794678
2.5788066387176514
2.669750928878784
2.7367537021636963
2.734037160873413
2.614459753036499
2.604853391647339
2.631786346435547
2.731353521347046
2.5957188606262207
2.527078628540039
2.5321249961853027
2.5650277137756348
2.582463502883911
2.6470608711242676
2.5752365589141846
2.6645567417144775
2.6521294116973877
2.675809860229492
2.596973419189453
2.7264177799224854
2.6959762573242188
2.6396827697753906
2.597627639770508
2.7534983158111572
2.551912307739258
2.593477964401245
2.641047239303589
2.719273090362549
2.630542755126953
2.5376696586608887
2.6295766830444336
2.5777087211608887
2.626631736755371
2.71602463722229
2.6021652221679688
2.610753059387207
2.647296190261841
2.6

In [52]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(m.generate(idx, max_new_token=200)[0].tolist()))


Qzeroules mickngbeat d:
ORDU'h w.
AELI arereer len INTh beare s ar in t nt aloust h omityor me'Benof trdendve do Wha t bld aindie in ar y; vermpe w alume, n s thlissthatof shilyono kenarelinotheted yb


In [55]:
torch.manual_seed(1010)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [61]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)

xbow2 = wei @ x

In [15]:
import torch.nn.functional as F

In [28]:
torch.manual_seed(1010)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
x.shape

#single head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x)

wei = q @ k.transpose(-2, -1)# (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
# xbow3 = wei @ x

out = wei @ v

#torch.allclose(xbow2, xbow3)

In [29]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6131, 0.3869, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0316, 0.2858, 0.6826, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6023, 0.0399, 0.1638, 0.1941, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7635, 0.0211, 0.0513, 0.0551, 0.1089, 0.0000, 0.0000, 0.0000],
        [0.2719, 0.1143, 0.2015, 0.2665, 0.0848, 0.0610, 0.0000, 0.0000],
        [0.0076, 0.0253, 0.8451, 0.0369, 0.0244, 0.0142, 0.0465, 0.0000],
        [0.0677, 0.0280, 0.1372, 0.3358, 0.0428, 0.2066, 0.0791, 0.1028]],
       grad_fn=<SelectBackward0>)

### Sels-Attention Block

In [30]:
torch.manual_seed(1010)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
x.shape

#single head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x)

wei = q @ k.transpose(-2, -1) * head_size ** -0.5# (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
# xbow3 = wei @ x

out = wei @ v

In [31]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5287, 0.4713, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2044, 0.3547, 0.4409, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3353, 0.1700, 0.2421, 0.2526, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3279, 0.1337, 0.1670, 0.1699, 0.2015, 0.0000, 0.0000, 0.0000],
        [0.1937, 0.1560, 0.1797, 0.1927, 0.1447, 0.1333, 0.0000, 0.0000],
        [0.0897, 0.1210, 0.2909, 0.1329, 0.1199, 0.1047, 0.1408, 0.0000],
        [0.1130, 0.0906, 0.1348, 0.1686, 0.1007, 0.1493, 0.1175, 0.1254]],
       grad_fn=<SelectBackward0>)